# Recognising handwritten digits with deep learning for smart AI application

Step 1: Imports and Setup


In [2]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


In [12]:
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
import gradio as gr

Step 2: Data Loading and Preprocessing

In [7]:
base_dir = "/content/sklearn_digits_by_folder (1).zip"

Create ImageDataGenerator

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# STEP 1: Unzip the dataset
zip_path = "/content/sklearn_digits_by_folder (1).zip"
extract_path = "/content/digits_folder"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# STEP 2: Define the base directory after extraction
base_dir = extract_path  # This should contain folders like 0, 1, ..., 9

# STEP 3: Data loading using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(32, 32),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(32, 32),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)


Found 1438 images belonging to 1 classes.
Found 359 images belonging to 1 classes.


Step 3: Model Building


In [15]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Step 4: Model Training

In [16]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)


Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.9099 - loss: 0.4832 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 1.0000 - loss: 3.4991e-11 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 1.0000 - loss: 7.2507e-10 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 1.0000 - loss: 6.8289e-09 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 1.0000 

Step 5: Evaluation


In [17]:
val_generator.reset()
y_true = val_generator.classes
y_pred = np.argmax(model.predict(val_generator), axis=1)
print("Classification Report:\n", classification_report(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))

12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       359

    accuracy                           1.00       359
   macro avg       1.00      1.00      1.00       359
weighted avg       1.00      1.00      1.00       359

Confusion Matrix:
 [[359]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


Step 6: Gradio Deployment


In [21]:
class_labels = list(train_generator.class_indices.keys())

def predict_digit(image):
    from PIL import Image
    img = Image.fromarray(image).convert('L').resize((32, 32))
    img = np.array(img) / 255.0
    img = img.reshape(1, 32, 32, 1)
    prediction = model.predict(img)
    return {class_labels[i]: float(prediction[0][i]) for i in range(10)}

import gradio as gr

interface = gr.Interface(
    fn=predict_digit,
    inputs=gr.Image(type="numpy", image_mode="L"),
    outputs=gr.Label(num_top_classes=3),
    title="Digit Recognition",
    description="Upload a handwritten digit image to get the predicted digit."
)

interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e7c4aa4fbf388d3251.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
